Based on v1's output and char_v6 notebook, implement a GPT2

```
transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.0.ln_1.weight torch.Size([768])
...
transformer.h.5.mlp.c_proj.bias torch.Size([768])
transformer.ln_f.weight torch.Size([768])
transformer.ln_f.bias torch.Size([768])
lm_head.weight torch.Size([50257, 768])
```

- wte: word token embedding -> maps input tokens to their corresponding vector representations
- wpe: word positional embedding
- c_attn: context attention -> linear transformation that projects the input embeddings into query, key, and value vectors for the self-attention. So the size is `[n_embd, 3 * n_embd]`
- c_proj: context projection -> linear transformation that projects the output of the self-attention back to the original embedding dimensionality. So the size is `[vocab_size, n_embd]`
- 3072 = 4 x 768
- 2304 = 3 x 768

In [1]:
import os
import numpy as np
from dataclasses import dataclass
import torch
from torch import nn
from torch.nn import functional as F
# from transformers import GPT2LMHeadModel
import matplotlib.pyplot as plt 

from utils import *; from boring_utils.utils import *
from data_structure import add_to_class

init_graph()
device = get_device()

# Hyperparameters

In [2]:
# bias True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster
@dataclass
class GPTConfig_small:
    block_size: int = 256
    vocab_size: int = 65
    n_layer: int = 6
    n_head: int = 6
    n_embd: int = 384
    dropout: float = 0.0
    bias: bool = True

# vocab_size: int = 50304: GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50257
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    dropout: float = 0.0
    bias: bool = True

# GPT

In [3]:
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, config.n_embd * 4, bias=config.bias)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(config.n_embd * 4, config.n_embd, bias=config.bias)
    
    def forward(self, x):
        x = self.c_fc(x)
        # x = F.gelu(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


# class Block(nn.Module):
#     '''
#     Attn is the 'reduce', MLP is the 'map' (no cross token ops)
#     Pytorch MHA's input shape is: sequence length first (or batch_first=True)
#     '''
#     def __init__(self, config):
#         super().__init__()
#         self.ln_1 = nn.LayerNorm(config.n_embd, bias=config.bias)
#         self.ln_2 = nn.LayerNorm(config.n_embd, bias=config.bias)
#         # self.attn = CasualSelfAttention(config)
#         self.attn = nn.MultiheadAttention(
#             config.n_embd, config.n_head)
#         self.mlp = MLP(config)
        
#         self.n_head = config.n_head
#         self.register_buffer(
#             "mask",
#             torch.triu(torch.ones(config.block_size, config.block_size), diagonal=1).bool()
#         )

#     def forward(self, x):
#         B, T, C = x.size()
#         # mask = torch.triu(torch.ones(T, T), diagonal=1).bool().to(x.device)
#         mask = self.mask[:T, :T]
#         x = x + self.attn(self.ln_1(x).transpose(0, 1), 
#                           self.ln_1(x).transpose(0, 1), 
#                           self.ln_1(x).transpose(0, 1), 
#                           attn_mask=mask)[0].transpose(0, 1)
#         x = x + self.mlp(self.ln_2(x))
#         return x


class Block(nn.Module):
    '''
    Attn is the 'reduce', MLP is the 'map' (no cross token ops)
    Pytorch MHA's input shape is: sequence length first (or batch_first=True)
    '''
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd, bias=config.bias)
        self.ln_2 = nn.LayerNorm(config.n_embd, bias=config.bias)
        # self.attn = CasualSelfAttention(config)
        self.attn = nn.MultiheadAttention(
            config.n_embd, config.n_head, batch_first=True)
        self.mlp = MLP(config)
        
        self.n_head = config.n_head
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(config.block_size, config.block_size), diagonal=1).bool()
        )

    def forward(self, x):
        B, T, C = x.size()
        # mask = torch.triu(torch.ones(T, T), diagonal=1).bool().to(x.device)
        mask = self.mask[:T, :T]
        x = x + self.attn(self.ln_1(x), self.ln_1(x), self.ln_1(x), attn_mask=mask)[0]
        x = x + self.mlp(self.ln_2(x))
        return x

In [6]:
class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd, bias=config.bias)
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
    
    def forward(self, idx):
        # idx shape: (B, T)
        B, T = idx.size()
        assert T <= self.config.block_size, f"input length {T} is longer than block size {self.config.block_size}"
        # pos = torch.arange(T, device=idx.device).unsqueeze(0).expand(B, T)
        pos = torch.arange(0, T, device=idx.device)  # shape: T
        pos_emb = self.transformer.wpe(pos)  # shape: (T, n_embd)
        tok_emb = self.transformer.wte(idx)  # shape: (B, T, n_embd)
        x = tok_emb + pos_emb

        for block in self.transformer.h:
            x = block(x)
        
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)  # shape: (B, T, Vocab Size)
        return logits

    @classmethod
    def from_pretrained(cls, model_type):
        '''https://youtu.be/l8pRSuU81PU?t=1830

        I insist using pytorch's MHA instead of HF. So I need a key_mapping dict.
        '''
        assert model_type in {'distilgpt2', 'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        config_args = {
            'distilgpt2':   dict(n_layer=6, n_head=12, n_embd=768),
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024),
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280),
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600),
        }[model_type]
        print("forcing vocab_size=50257, block_size=1024, bias=True")
        config_args.update(vocab_size=50257, block_size=1024, bias=True)

        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)

        key_mapping = {
            'transformer.wte.weight': 'transformer.wte.weight',
            'transformer.wpe.weight': 'transformer.wpe.weight',
            'transformer.ln_f.weight': 'transformer.ln_f.weight',
            'transformer.ln_f.bias': 'transformer.ln_f.bias',
            'lm_head.weight': 'lm_head.weight',
        }

        for i in range(config.n_layer):
            layer_mapping = {
                f'transformer.h.{i}.ln_1.weight': f'transformer.h.{i}.ln_1.weight',
                f'transformer.h.{i}.ln_1.bias': f'transformer.h.{i}.ln_1.bias',
                f'transformer.h.{i}.ln_2.weight': f'transformer.h.{i}.ln_2.weight',
                f'transformer.h.{i}.ln_2.bias': f'transformer.h.{i}.ln_2.bias',
                f'transformer.h.{i}.mlp.c_fc.weight': f'transformer.h.{i}.mlp.c_fc.weight',
                f'transformer.h.{i}.mlp.c_fc.bias': f'transformer.h.{i}.mlp.c_fc.bias',
                f'transformer.h.{i}.mlp.c_proj.weight': f'transformer.h.{i}.mlp.c_proj.weight',
                f'transformer.h.{i}.mlp.c_proj.bias': f'transformer.h.{i}.mlp.c_proj.bias',
            }
            key_mapping.update(layer_mapping)

            # copy attn weights
            with torch.no_grad():
                c_attn_weight = model_hf.state_dict()[f'transformer.h.{i}.attn.c_attn.weight'].t()  # (3*n_embd, n_embd)
                c_attn_bias = model_hf.state_dict()[f'transformer.h.{i}.attn.c_attn.bias']  # (3*n_embd,)
                model.transformer.h[i].attn.in_proj_weight.copy_(c_attn_weight)
                model.transformer.h[i].attn.in_proj_bias.copy_(c_attn_bias)
                out_proj_weight = model_hf.state_dict()[f'transformer.h.{i}.attn.c_proj.weight'].t()
                out_proj_bias = model_hf.state_dict()[f'transformer.h.{i}.attn.c_proj.bias']
                model.transformer.h[i].attn.out_proj.weight.copy_(out_proj_weight)
                model.transformer.h[i].attn.out_proj.bias.copy_(out_proj_bias)

        for k in key_mapping:
            if k in model_hf.state_dict():
                if 'mlp' in k and k.endswith('.weight'):
                    model.state_dict()[key_mapping[k]].copy_(model_hf.state_dict()[k].t())
                else:
                    model.state_dict()[key_mapping[k]].copy_(model_hf.state_dict()[k])

        return model


num_return_sequences = 4
max_length = 30

# model = GPT.from_pretrained('distilgpt2')
model = GPT.from_pretrained('gpt2')
model.eval()
model.to(device)

loading weights from pretrained gpt: gpt2
forcing vocab_size=50257, block_size=1024, bias=True


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

# Gen

Reproduce HF

```python
generator = pipeline('text-generation', model='gpt2')
generator("Hello, I'm a horny language model,", max_length=30, num_return_sequences=5)
```

In [5]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello, I'm a horny language model,")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
x = tokens.to(device)

while x.size(1) < max_length:
    with torch.no_grad():
        logits = model(x)  # (B, T, vocab_size)

        # take the logits at the last position
        logits = logits[:, -1, :]  # (B, vocab_size)

        # get the probabilities
        probs = F.softmax(logits, dim=-1)

        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        # turn to zero for all indices below the top-k
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)

        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        # [Multinomial distribution - Wikipedia](https://en.wikipedia.org/wiki/Multinomial_distribution)
        ix = torch.multinomial(topk_probs, 1)  # (B, 1)

        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix)  # (B, 1)

        # append to the sequence
        x = torch.cat((x, xcol), dim=1)


# print the generated text
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(f"> {decoded}")

> Hello, I'm a horny language model, one that would have nothing to do with the normal daily life of my life. It is a sad state
> Hello, I'm a horny language model, I don't take any risks... You're right, I'm not a good looking guy just a little
> Hello, I'm a horny language model, and the video game has made me happy! It's an almost 20+ minute video that I made because
> Hello, I'm a horny language model, I'm obsessed with my fetish, this is like the worst I've ever been!! I'm the first
